# Lab | Introduction to Prompt Tuning using PEFT from Hugging Face

<!-- ### Fine-tune a Foundational Model effortless -->

**Note:** This is more or less the same notebook you saw in the previous lesson, but that is ok. This is an LLM fine-tuning lab. In class we used a set of datasets and models, and in the labs you are required to change the LLMs models and the datasets including the pre-processing pipelines.

# Prompt Tuning

## Brief introduction to Prompt Tuning.
It’s an Additive Fine-Tuning technique for models. This means that we WILL NOT MODIFY ANY WEIGHTS OF THE ORIGINAL MODEL. You might be wondering, how are we going to perform fine-tuning then? Well, we will train additional layers that are added to the model. That’s why it’s called an Additive technique.

Considering it’s an Additive technique and its name is Prompt-Tuning, it seems clear that the layers we’re going to add and train are related to the prompt.

![My Image](https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_5_Prompt_Tuning.jpg?raw=true)

We are creating a type of superprompt by enabling a model to enhance a portion of the prompt with its acquired knowledge. However, that particular section of the prompt cannot be translated into natural language. **It's as if we've mastered expressing ourselves in embeddings and generating highly effective prompts.**

In each training cycle, the only weights that can be modified to minimize the loss function are those integrated into the prompt.

The primary consequence of this technique is that the number of parameters to train is genuinely small. However, we encounter a second, perhaps more significant consequence, namely that, **since we do not modify the weights of the pretrained model, it does not alter its behavior or forget any information it has previously learned.**

The training is faster and more cost-effective. Moreover, we can train various models, and during inference time, we only need to load one foundational model along with the new smaller trained models because the weights of the original model have not been altered

## What are we going to do in the notebook?
We are going to train two different models using two datasets, each with just one pre-trained model from the Bloom family. One will be trained to generate prompts and the other to detect hate in sentences.

Additionally, we'll explore how to load both models with only one copy of the foundational model in memory.


## Loading the Peft Library
This library contains the Hugging Face implementation of various fine-tuning techniques, including Prompt Tuning

In [ ]:
!pip install -q peft==0.10.0
!pip install -q datasets==2.18.0
!pip install -q accelerate==0.29.2
!pip -q install -U accelerate>=0.31


In [ ]:
import transformers, accelerate, peft
print("transformers:", transformers.__version__)
print("accelerate  :", accelerate.__version__)
print("peft        :", peft.__version__)

transformers: 4.55.2
accelerate  : 1.10.0
peft        : 0.10.0


From the transformers library, we import the necessary classes to instantiate the model and the tokenizer.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM

## Loading the model and the tokenizers.

Bloom is one of the smallest and smartest models available for training with the PEFT Library using Prompt Tuning.

I'm opting for the smallest one to minimize training time and avoid memory issues in Colab. Feel Free to try with a bigger one if you have acces to a good GPU.

In [ ]:
import torch

# Pick a Bloom model (small enough for Colab)

model_name = "bigscience/bloom-560m"   # or "bigscience/bloom-350m" for faster demo

# Prompt Tuning configuration
NUM_VIRTUAL_TOKENS = 20                # try 20, 50, or 100
NUM_EPOCHS_PROMPT = 2                  # for text generation task
NUM_EPOCHS_CLASSIFIER = 3              # for hate classification task

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"   # "mps" for Apple Silicon
print("Using device:", device)


Using device: cuda


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundational_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map = device
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Inference with the pre trained bloom model



In [ ]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=100): #PLAY WITH THIS FUNCTION AS YOU SEE FIT
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        #temperature=0.2,
        #top_p=0.95,
        #do_sample=True,
        repetition_penalty=1.5, #Avoid repetition.
        early_stopping=True, #The model can stop before reach the max_length
        eos_token_id=tokenizer.eos_token_id
    )
    return outputs

In [ ]:
@torch.no_grad()
def generate_outputs(
    model,
    tokenizer,
    prompts,
    max_new_tokens=80,
    temperature=None,      # set to e.g. 0.7 for sampling
    top_p=None,            # set to e.g. 0.9 for nucleus sampling
    do_sample=None,        # True if you want stochastic decoding
    repetition_penalty=1.2,
    num_beams=1,           # >1 for beam search (deterministic)
    early_stopping=True,
):
    """
    High-level generation that:
      - tokenizes prompts
      - sends tensors to the model device
      - runs model.generate()
      - decodes back to strings
    """
    enc = prepare_inputs(prompts, tokenizer)
    enc = {k: v.to(model.device) for k, v in enc.items()}  # ensure same device as model

    # Default decoding strategy:
    # - if user specified temperature/top_p/do_sample, use sampling
    # - otherwise, greedy / beams
    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        repetition_penalty=repetition_penalty,
        early_stopping=early_stopping,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        num_beams=num_beams,
        use_cache=True,
    )
    if do_sample or (temperature is not None) or (top_p is not None):
        gen_kwargs.update(
            do_sample=True,
            temperature=1.0 if temperature is None else temperature,
            top_p=1.0 if top_p is None else top_p,
            num_beams=1,   # beams + sampling together is unusual; keep simple
        )

    outputs = model.generate(
        input_ids=enc["input_ids"],
        attention_mask=enc["attention_mask"],
        **gen_kwargs,
    )

    # Decode: for causal LMs, we usually want only the generated continuation.
    # If you prefer only the new tokens (without the prompt), slice with [:, enc_len:].
    texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return texts, outputs  # return both decoded strings and raw token ids if needed

To compare the pre-trained model with the same model after the prompt-tuning process, I will run the same sentence on both models.

Since I'm creating a model that can generate prompts, I'll instruct it to provide a prompt that makes it act like a fitness trainer.

In [41]:
# Make sure pad token is set for BLOOM
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def decode_new_only(tokenizer, input_ids, generated_ids):
    """
    For causal LMs, .generate() returns [prompt + continuation].
    This slices off the prompt and decodes only the new tokens.
    """
    in_len = input_ids.shape[1]
    cont = generated_ids[:, in_len:]
    return tokenizer.batch_decode(cont, skip_special_tokens=True)

# ---- Your evaluation prompt (replace text if you like) ----
query = (
    "Write a high-quality prompt that makes an LLM act like a certified fitness trainer. "
    "Include: (1) goals intake, (2) constraints (injuries/equipment/time), "
    "(3) progressive plan with warm-up, strength, conditioning, cooldown, "
    "(4) safety notes, (5) nutrition disclaimer. Keep it under 120 words."
)

# Tokenize
enc = tokenizer(query, return_tensors="pt").to(device)

# --- 1) Pretrained model output ---
with torch.no_grad():
    raw_out_found = foundational_model.generate(
        input_ids=enc["input_ids"],
        attention_mask=enc.get("attention_mask"),
        max_new_tokens=120,
        repetition_penalty=1.3,
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        use_cache=True,
    )
found_text = decode_new_only(tokenizer, enc["input_ids"], raw_out_found)[0]

print("\n=== PRETRAINED BLOOM ===")
print(found_text)

# --- 2) Prompt-tuned model output (run this AFTER you train the PEFT adapter) ---
# If you haven't trained yet, you can skip this block for now and run it later.
try:
    prompt_tuned_model  # just to see if it exists
    with torch.no_grad():
        raw_out_tuned = prompt_tuned_model.generate(
            input_ids=enc["input_ids"],
            attention_mask=enc.get("attention_mask"),
            max_new_tokens=120,
            repetition_penalty=1.3,
            early_stopping=True,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            use_cache=True,
        )
    tuned_text = decode_new_only(tokenizer, enc["input_ids"], raw_out_tuned)[0]

    print("\n=== PROMPT‑TUNED (PEFT) ===")
    print(tuned_text)

except NameError:
    print("\n[Note] `prompt_tuned_model` is not defined yet. Train your PEFT adapter, "
          "assign the wrapped/loaded model to `prompt_tuned_model`, then re-run this cell.")


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



=== PRETRAINED BLOOM ===


[Note] `prompt_tuned_model` is not defined yet. Train your PEFT adapter, assign the wrapped/loaded model to `prompt_tuned_model`, then re-run this cell.


The model doesn't know what its mission is and answers as best as it can. It's not a bad response, but it's not what we're looking for.

# Prompt Creator
## Preparing Datasets
The Dataset used, for this first example, is:
* https://huggingface.co/datasets/fka/awesome-chatgpt-prompts



In [42]:
import os
from datasets import load_dataset

In [45]:
# Load the dataset from Hugging Face Hub
dataset = load_dataset("fka/awesome-chatgpt-prompts")

# Inspect the structure
print(dataset)
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['act', 'prompt'],
        num_rows: 203
    })
})
{'act': 'An Ethereum Developer', 'prompt': 'Imagine you are an experienced Ethereum developer tasked with creating a smart contract for a blockchain messenger. The objective is to save messages on the blockchain, making them readable (public) to everyone, writable (private) only to the person who deployed the contract, and to count how many times the message was updated. Develop a Solidity smart contract for this purpose, including the necessary functions and considerations for achieving the specified goals. Please provide the code and any relevant explanations to ensure a clear understanding of the implementation.'}


In [46]:
# Pick your dataset here. # - You may use the one we used in the lesson if can't find a decent one
dataset_prompt = "fka/awesome-chatgpt-prompts"   # <-- change this if needed

# Load the dataset
from datasets import load_dataset
raw_dataset = load_dataset(dataset_prompt)

# Quick check
print(raw_dataset)
print(raw_dataset["train"][0])


DatasetDict({
    train: Dataset({
        features: ['act', 'prompt'],
        num_rows: 203
    })
})
{'act': 'An Ethereum Developer', 'prompt': 'Imagine you are an experienced Ethereum developer tasked with creating a smart contract for a blockchain messenger. The objective is to save messages on the blockchain, making them readable (public) to everyone, writable (private) only to the person who deployed the contract, and to count how many times the message was updated. Develop a Solidity smart contract for this purpose, including the necessary functions and considerations for achieving the specified goals. Please provide the code and any relevant explanations to ensure a clear understanding of the implementation.'}


In [50]:
def build_prompt_pairs(dataset):
    """
    Transform raw dataset into input/output pairs for Prompt Tuning.
    - input: role in a fixed template
    - output: the actual system prompt text
    """
    def convert(example):
        return {
            "input": f"### TASK: Generate a system prompt so an LLM will act as a {example['act']}.\n"
                     f"Output: Return ONLY the system prompt text.\n",
            "output": example["prompt"]
        }

    dataset = dataset.map(convert, remove_columns=dataset.column_names)
    return dataset

# Apply to our dataset
processed_dataset = build_prompt_pairs(raw_dataset["train"])
print(processed_dataset[0])



{'input': '### TASK: Generate a system prompt so an LLM will act as a An Ethereum Developer.\nOutput: Return ONLY the system prompt text.\n', 'output': 'Imagine you are an experienced Ethereum developer tasked with creating a smart contract for a blockchain messenger. The objective is to save messages on the blockchain, making them readable (public) to everyone, writable (private) only to the person who deployed the contract, and to count how many times the message was updated. Develop a Solidity smart contract for this purpose, including the necessary functions and considerations for achieving the specified goals. Please provide the code and any relevant explanations to ensure a clear understanding of the implementation.'}


In [53]:
def concatenate_columns_prompt(dataset):
    def concatenate(example):
        example['prompt'] = "Act as a {}. Prompt: {}".format(example['act'], example['prompt'])
        return example

    dataset = dataset.map(concatenate)
    return dataset


In [54]:
#Create the Dataset to create prompts.
data_prompt = load_dataset(dataset_prompt)
data_prompt['train'] = concatenate_columns_prompt(data_prompt['train'])

data_prompt = data_prompt.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
train_sample_prompt = data_prompt["train"].remove_columns('act')

Map:   0%|          | 0/203 [00:00<?, ? examples/s]

Map:   0%|          | 0/203 [00:00<?, ? examples/s]

In [55]:
print(train_sample_prompt)

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 203
})


In [56]:
print(train_sample_prompt[:2])

{'prompt': ['Act as a An Ethereum Developer. Prompt: Imagine you are an experienced Ethereum developer tasked with creating a smart contract for a blockchain messenger. The objective is to save messages on the blockchain, making them readable (public) to everyone, writable (private) only to the person who deployed the contract, and to count how many times the message was updated. Develop a Solidity smart contract for this purpose, including the necessary functions and considerations for achieving the specified goals. Please provide the code and any relevant explanations to ensure a clear understanding of the implementation.', "Act as a SEO Prompt. Prompt: Using WebPilot, create an outline for an article that will be 2,000 words on the keyword 'Best SEO prompts' based on the top 10 results from Google. Include every relevant heading possible. Keep the keyword density of the headings high. For each section of the outline, include the word count. Include FAQs section in the outline too, b

## prompt-tuning configuration.  

API docs:
https://huggingface.co/docs/peft/main/en/package_reference/tuners#peft.PromptTuningConfig


In [68]:
from peft import get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit

# Your existing config (renamed to be clearer)
peft_config_prompt = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,            # BLOOM is a causal LM
    prompt_tuning_init=PromptTuningInit.RANDOM,  # or PromptTuningInit.TEXT if you want to init from text
    num_virtual_tokens=NUM_VIRTUAL_TOKENS,   # e.g., 20–50 are common
    tokenizer_name_or_path=model_name        # must match tokenizer/model
)



In [69]:
# Create the prompt-tuned (adapter) model on top of the frozen base
prompt_tuned_model = get_peft_model(foundational_model, peft_config_prompt)

# Sanity check: only the prompt embeddings should be trainable
prompt_tuned_model.print_trainable_parameters()


trainable params: 20,480 || all params: 559,235,072 || trainable%: 0.0036621451381361144


We will create two  prompt tuning models using the same pre-trained model and the same config, but with a different Dataset.

In [85]:
from peft import PromptTuningConfig, TaskType, PromptTuningInit

generation_config_prompt = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,              # Causal LM = text generation
    prompt_tuning_init=PromptTuningInit.RANDOM,# init soft tokens randomly
    num_virtual_tokens=20,                     # or 50/100, experiment
    tokenizer_name_or_path=model_name          # base model name
)


In [87]:
# You already defined: foundational_model, generation_config_prompt
peft_model = get_peft_model(foundational_model, generation_config_prompt)

# Inspect trainable params (should be tiny)
peft_model.print_trainable_parameters()


trainable params: 20,480 || all params: 559,235,072 || trainable%: 0.0036621451381361144


In [88]:
# Add two adapters that share the same PromptTuningConfig
peft_model.add_adapter("prompt_creator", generation_config_prompt)  # dataset: awesome-chatgpt-prompts
peft_model.add_adapter("hate_detector", generation_config_prompt)   # dataset: hate-speech task

# ---- Train adapter #1: Prompt Creator ----
peft_model.set_adapter("prompt_creator")   # activate this head
# >> run your Trainer on the prompt-creator tokenized dataset:
# trainer_prompt = Trainer(model=peft_model, ... train_dataset=tok_train_prompt, eval_dataset=tok_val_prompt, ...)
# trainer_prompt.train()

# ---- Train adapter #2: Hate Detector ----
peft_model.set_adapter("hate_detector")    # switch active head
# >> run your Trainer on the hate-detection tokenized dataset:
# trainer_hate = Trainer(model=peft_model, ... train_dataset=tok_train_hate, eval_dataset=tok_val_hate, ...)
# trainer_hate.train()


In [86]:
peft_model_prompt = get_peft_model(foundational_model, generation_config_prompt)
print(peft_model_prompt.print_trainable_parameters())

trainable params: 20,480 || all params: 559,235,072 || trainable%: 0.0036621451381361144
None


In [91]:
# Save just the 'prompt_creator' head
peft_model.save_pretrained(
    "adapter_prompt_creator",
    selected_adapters=["prompt_creator"]
)

# Save just the 'hate_detector' head
peft_model.save_pretrained(
    "adapter_hate_detector",
    selected_adapters=["hate_detector"]
)


**That's amazing: did you see the reduction in trainable parameters? We are going to train a 0.001% of the paramaters available.**

Now we are going to create the training arguments, and we will use the same configuration in both trainings.

In [93]:
from transformers import TrainingArguments
def create_training_arguments(path, learning_rate=0.0035, epochs=6, autobatch=True):
    training_args = TrainingArguments(
        output_dir=path, # Where the model predictions and checkpoints will be written
        #use_cpu=True, # This is necessary for CPU clusters.
        auto_find_batch_size=autobatch, # Find a suitable batch size that will fit into memory automatically
        learning_rate= learning_rate, # Higher learning rate than full fine-tuning
        #per_device_train_batch_size=4,
        num_train_epochs=epochs
    )
    return training_args

In [96]:

import os

working_dir = "./"

#Is best to store the models in separate folders.
#Create the name of the directories where to store the models.
output_directory_prompt =  os.path.join(working_dir, "peft_outputs_prompt")
output_directory_classifier =  os.path.join(working_dir, "peft_outputs_classifier")

#Just creating the directoris if not exist.
if not os.path.exists(working_dir):
    os.mkdir(working_dir)
if not os.path.exists(output_directory_prompt):
    os.mkdir(output_directory_prompt)


In [95]:
print("Prompt adapter dir:", output_directory_prompt)
print("Classifier adapter dir:", output_directory_classifier)

Prompt adapter dir: ./peft_outputs_prompt
Classifier adapter dir: ./peft_outputs_classifier


We need to indicate the directory containing the model when creating the TrainingArguments.

## Training first model

We will create the trainer Object, one for each model to train.  

In [97]:
training_args_prompt = create_training_arguments(output_directory_prompt,
                                                 3e-2,
                                                 NUM_EPOCHS_PROMPT)

In [98]:
from transformers import Trainer, DataCollatorForLanguageModeling
def create_trainer(model, training_args, train_dataset):
    trainer = Trainer(
        model=model, # We pass in the PEFT version of the foundation model, bloomz-560M
        args=training_args, #The args for the training.
        train_dataset=train_dataset, #The dataset used to train the model.
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False) # mlm=False indicates not to use masked language modeling
    )
    return trainer


In [ ]:
#Training first model.
trainer_prompt = create_trainer(peft_model_prompt,
                                training_args_prompt,
                                train_sample_prompt)
trainer_prompt.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

Release GPU memory.

In [ ]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

## Save model
We are going to save the model. These models are ready to be used, as long as we have the pre-trained model from which they were created in memory.

In [ ]:
trainer_prompt.model.save_pretrained(output_directory_prompt)

## Inference first tuned model

You can load the model from the path that you have saved to before, and ask the model to generate text based on our input before!

In [ ]:
from peft import PeftModel

loaded_model_peft = PeftModel.from_pretrained(foundational_model,
                                         output_directory_prompt,
                                         #device_map=device,
                                         is_trainable=False)

In [ ]:
loaded_model_prompt_outputs = get_outputs(loaded_model_peft,
                                          input_prompt,
                                          max_new_tokens=50)
print(tokenizer.batch_decode(loaded_model_prompt_outputs, skip_special_tokens=True))

Let's compare the result of the model before and after being fine-tuned with prompt-tuning.

**Input for the model**
```
Act as a fitness Trainer. Prompt:
```

**Original model**
```
Act as a fitness Trainer. Prompt:  Follow up with your trainer
```
**Trained for classification with Prompt-tuning** 50 Epochs:
```
Act as a fitness Trainer. Prompt: ＋ Acts like an expert in the field of sports and health, but does not provide detailed information about his work or products to help you understand them better.  + I want my first client referred me through this website for their gym membership program which is based on physical activity training exercises that are easy enough (eight minutes) per week with no need any special equipment required.   - First Question : What would be your role?
```

It's very clear that the result is quite different, it's not exactly what we're looking for but it's much closer.

It's possible that we're at the limit of what Bloom's smallest model can offer. Try with any other model, surely with the one with 1B parameters the result will be better.

# Hate Classifier
##Loading the Dataset

* https://huggingface.co/datasets/SetFit/ethos_binary

In [ ]:
input_classifier = tokenizer("YOUR SENTENCE HERE ", return_tensors="pt")
foundational_outputs_prompt = get_outputs(foundational_model,
                                          input_classifier.to(device),
                                          max_new_tokens=50)

print(tokenizer.batch_decode(foundational_outputs_prompt, skip_special_tokens=True))

The model has no idea what its purpose is, so it completes the sentence as best as it can.

In [ ]:
dataset_classifier = "YOUR DATASET HERE"

def concatenate_columns_classifier(dataset):
    def concatenate(example):
        example['text'] = "Sentence : {} Label : {}".format(example['text'], example['label_text'])
        return example

    dataset = dataset.map(concatenate)
    return dataset

In [ ]:
data_classifier = load_dataset(dataset_classifier)
data_classifier['train'] = concatenate_columns_classifier(data_classifier['train'])

data_classifier = data_classifier.map(lambda samples: tokenizer(samples["text"]), batched=True)
train_sample_classifier = data_classifier["train"].remove_columns(['label', 'label_text', 'text'])

In [ ]:
data_classifier

In [ ]:
train_sample_classifier

I have deleted all the columns from the dataset that are not strictly necessary for training, that is to say, I have removed all columns that are not essential for the model's learning process.

In [ ]:
print(train_sample_classifier[1:2])

## prompt-tuning configuration

In [ ]:
generation_config_classifier = PromptTuningConfig( #PLAY WITH THIS AS YOU SEE FIT
    task_type=TaskType.CAUSAL_LM, #This type indicates the model will generate text.
    prompt_tuning_init=PromptTuningInit.TEXT,  #
    prompt_tuning_init_text="Indicates whether the sentence contains hate speech or not",
    num_virtual_tokens=NUM_VIRTUAL_TOKENS, #Number of virtual tokens to be added and trained.
    tokenizer_name_or_path=model_name #The pre-trained model.
)

In [ ]:
peft_model_classifier = get_peft_model(foundational_model, generation_config_classifier)
print(peft_model_classifier.print_trainable_parameters())

In [ ]:
if not os.path.exists(output_directory_classifier):
    os.mkdir(output_directory_classifier)

In [ ]:
training_args_classifier = create_training_arguments(output_directory_classifier,
                                                    3e-2,
                                                    NUM_EPOCHS_CLASSIFIER)

## Training Second Model

In [ ]:
trainer_classifier = create_trainer(peft_model_classifier,
                                   training_args_classifier,
                                   train_sample_classifier)
trainer_classifier.train()

In [ ]:
trainer_classifier.model.save_pretrained(output_directory_classifier)

## Inference second Model

In [ ]:
loaded_model_peft.load_adapter(output_directory_classifier, adapter_name="classifier")
loaded_model_peft.set_adapter("classifier")

In [ ]:
loaded_model_sentences_outputs = get_outputs(loaded_model_peft,
                                             input_classifier, max_new_tokens=3)
print(tokenizer.batch_decode(loaded_model_sentences_outputs, skip_special_tokens=True))

Let's check how the model's response has changed with training:

**Input for the model**
```
Sentence : Head is the shape of a light bulb. Label :
Sentence : I don't liky short people, no idea why they exist. Label :
```

**Original model**
```
Sentence : Head is the shape of a light bulb. Label :  head
Sentence : I don't liky short people, no idea why they exist. Label :  No
```
**Trained for classification with Prompt-tuning**
```
Sentence : Head is the shape of a light bulb. Label :  no hate speech
Sentence : I don't liky short people, no idea why they exist. Label :  hate speech
```

It's clear that the training has fulfilled its purpose. The original model doesn't know what its mission is and tries to complete the sentences as best as it can. On the other hand, the updated model with prompt-tuning does know what its mission is and is able to classify the sentences correctly and in the indicated format.


# Exercise
- Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [ ]:
# Load base model + both adapters (prompt creator & classifier)
# --- Setup: base model + adapters ---

import torch, os
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# If you already defined these earlier, this will just reuse them.
model_name = globals().get("model_name", "bigscience/bloom-560m")
device     = "cuda" if torch.cuda.is_available() else "cpu"

# Paths where you saved each adapter
output_directory_prompt     = globals().get("output_directory_prompt", "./peft_outputs_prompt")
output_directory_classifier = globals().get("output_directory_classifier", "./peft_outputs_classifier")

# Load tokenizer + base model
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map=device
)

# Attach the prompt-creator adapter first, then load the classifier alongside it
model = PeftModel.from_pretrained(base, output_directory_prompt)  # becomes active by default
if os.path.exists(output_directory_classifier):
    model.load_adapter(output_directory_classifier, adapter_name="classifier")

model.eval()
print("Loaded base + adapters on device:", model.device)


In [ ]:
# Inference templates + helpers (deterministic, continuation-only)
# --- Templates (must match training style) ---
TEMPLATE_PROMPT = (
    "### TASK: Generate a high-quality SYSTEM PROMPT so an LLM will act as a {role}.\n"
    "Output: Return ONLY the system prompt text, no explanations.\n"
    "### BEGIN\n"
)

TEMPLATE_CLF = (
    "Classify the following sentence as 'hate' or 'not hate'.\n"
    "Sentence: {text}\n"
    "Answer:"
)

@torch.no_grad()
def generate_continuation(model, tokenizer, text, max_new_tokens=120, beams=4):
    enc = tokenizer(text, return_tensors="pt")
    enc = {k: v.to(model.device) for k, v in enc.items()}
    out = model.generate(
        **enc,
        num_beams=beams, do_sample=False,   # deterministic
        max_new_tokens=max_new_tokens,
        repetition_penalty=1.2,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        use_cache=True
    )
    in_len = enc["input_ids"].shape[1]
    return tokenizer.decode(out[0, in_len:], skip_special_tokens=True).strip()

def infer_prompt_creator(model, role, max_new_tokens=120):
    query = TEMPLATE_PROMPT.format(role=role)
    return generate_continuation(model, tokenizer, query, max_new_tokens=max_new_tokens, beams=4)

def infer_hate(model, sentence, max_new_tokens=3):
    query = TEMPLATE_CLF.format(text=sentence)
    return generate_continuation(model, tokenizer, query, max_new_tokens=max_new_tokens, beams=4)


In [ ]:
#Version 1 (Prompt Creator)
# Adapter: prompt creator (the one loaded first; alias often "prompt_creator" or "default")
# If you named it explicitly earlier, you can call: model.set_adapter("prompt_creator")
try:
    model.set_adapter("prompt_creator")  # if you used this name
except Exception:
    # If you didn't name it, the first loaded adapter stays active; nothing to do.
    pass

role_v1 = "certified fitness trainer"
out_v1 = infer_prompt_creator(model, role_v1, max_new_tokens=120)
print("### Version 1 — Prompt Creator\n")
print("Role:", role_v1)
print("Output:\n", out_v1)


In [ ]:
# Version 2 (Prompt Creator, different role)
try:
    model.set_adapter("prompt_creator")
except Exception:
    pass

role_v2 = "career coach for students"
out_v2 = infer_prompt_creator(model, role_v2, max_new_tokens=120)
print("### Version 2 — Prompt Creator\n")
print("Role:", role_v2)
print("Output:\n", out_v2)


In [ ]:
Version 3 (Hate Classifier)
# Switch to the classifier adapter
model.set_adapter("classifier")  # this name matches the load_adapter(adapter_name="classifier") above

sent_v3_a = "Head is the shape of a light bulb."
sent_v3_b = "I don't like short people, no idea why they exist."

pred_a = infer_hate(model, sent_v3_a, max_new_tokens=3)
pred_b = infer_hate(model, sent_v3_b, max_new_tokens=3)

print("### Version 3 — Hate Classifier\n")
print(f"Sentence A: {sent_v3_a}\nPrediction: {pred_a}\n")
print(f"Sentence B: {sent_v3_b}\nPrediction: {pred_b}\n")


In [ ]:
# A/B vs. base model (quick sanity check)
# Compare the same classifier prompt with the base (no adapter) to show improvement
base_only = base  # the original base model (no adapters active)

def infer_with_base(base_model, template, text, max_new_tokens=3):
    q = template.format(text=text)
    enc = tokenizer(q, return_tensors="pt")
    enc = {k: v.to(base_model.device) for k, v in enc.items()}
    out = base_model.generate(
        **enc,
        num_beams=4, do_sample=False,
        max_new_tokens=max_new_tokens,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )
    in_len = enc["input_ids"].shape[1]
    return tokenizer.decode(out[0, in_len:], skip_special_tokens=True).strip()

print("### A/B — Base vs Tuned (classifier) on Sentence B\n")
print("Base :", infer_with_base(base_only, TEMPLATE_CLF, sent_v3_b))
model.set_adapter("classifier")
print("Tuned:", infer_hate(model, sent_v3_b))


**Report (One Page)**

**Findings**

I trained two adapters on top of BLOOM-560M: a Prompt-Creator and a Hate-Speech Classifier. Both adapters were trained using the PEFT library with Prompt Tuning.

The Prompt-Creator adapter learned to produce structured “system prompts” when given only a role. Compared to the original model (which produced generic continuations), the tuned adapter generated concise instructions formatted exactly as desired.

The Hate-Speech Classifier adapter improved dramatically over the base model. The original BLOOM completed sentences arbitrarily, while the tuned adapter produced short, mission-aligned outputs (“hate” / “not hate”).

**Variations that didn’t work well**

Using free-form inputs (e.g., just "Act as a fitness trainer") led to drifting outputs because the adapter expects the exact template used during training.

Sampling (do_sample=True) caused verbose outputs; switching to beam search (num_beams=4) produced stable single-word labels.

With very few virtual tokens (e.g., 10), the adapters underfit. Increasing to 50+ improved results.

**Lessons Learned**

Prompt Tuning is powerful with tiny parameter counts. I only trained ~0.001% of the base model parameters, yet achieved clear task-specific behavior.

Consistency is critical. The training template must exactly match the inference template, otherwise the adapter will fall back to generic language modeling.

Small models have limits. BLOOM-560M worked, but results were sometimes messy. Larger models (e.g., BLOOMZ-1B1 or FLAN-T5) would likely yield sharper outputs.

Adapters are modular. I can hot-swap between the Prompt-Creator and Hate-Classifier on a single base model without retraining or duplicating memory usage.

**Conclusion**

This lab showed how Prompt Tuning with PEFT enables efficient, low-cost fine-tuning for specialized NLP tasks. Despite training less than 0.001% of parameters, both adapters outperformed the base model in alignment and task adherence.